In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
Compozit = pd.read_excel('C:/Users/Владимир/OneDrive/Desktop/работа/Обучение/ВКР/Датасет для ВКР_композиты/Compozit_int.xlsx') 

In [6]:
Compozit

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2","Угол нашивки, град",Шаг нашивки,Плотность нашивки
0,1.857143,2030.000000,738.736842,50.000000,23.750000,284.615385,210.000000,70.000000,3000.000000,220.000000,0,4,60
1,1.857143,2030.000000,738.736842,129.000000,21.250000,300.000000,210.000000,70.000000,3000.000000,220.000000,0,5,47
2,2.771331,2030.000000,753.000000,111.860000,22.267857,284.615385,210.000000,70.000000,3000.000000,220.000000,0,5,57
3,2.767918,2000.000000,748.000000,111.860000,22.267857,284.615385,210.000000,70.000000,3000.000000,220.000000,0,5,60
4,2.569620,1910.000000,807.000000,111.860000,22.267857,284.615385,210.000000,70.000000,3000.000000,220.000000,0,5,70
5,2.561475,1900.000000,535.000000,111.860000,22.267857,284.615385,380.000000,75.000000,1800.000000,120.000000,0,7,47
6,3.557018,1930.000000,889.000000,129.000000,21.250000,300.000000,380.000000,75.000000,1800.000000,120.000000,0,7,57
7,3.532338,2100.000000,1421.000000,129.000000,21.250000,300.000000,1010.000000,78.000000,2000.000000,300.000000,0,7,60
8,2.919678,2160.000000,933.000000,129.000000,21.250000,300.000000,1010.000000,78.000000,2000.000000,300.000000,0,7,70
9,2.877358,1990.000000,1628.000000,129.000000,21.250000,300.000000,1010.000000,78.000000,2000.000000,300.000000,0,9,47


In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression,GammaRegressor, ARDRegression
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor,BaggingRegressor
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

In [9]:
y_1 = Compozit['Модуль упругости при растяжении, ГПа']
y_2 = Compozit['Прочность при растяжении, МПа']
Compozit.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Соотношение матрица-наполнитель'], axis=1, inplace=True)
X = Compozit.copy()

Т.к. данные очень сильно отличаются, и они не следуют нормальному распределению, то для будет создано 2 вида для каждой модели. \
Одна из них будет с нормализатором MinMaxScaler

**1. Подбор модели для переменной 'Модуль упругости при растяжении, ГПа'.**

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y_1, test_size=0.3, random_state=42)

In [184]:
def my_best_model(X, y_1):
    algos = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}
    },
    'GammaRegressor': {
        'model': GammaRegressor(),
        'params': {}
   },
    'ARDRegression': {
        'model': ARDRegression(),
        'params': {'tol': [1e-3, 1e-2]}
    },
    'AdaBoostRegressor': {
        'model': AdaBoostRegressor(),
        'params': {'n_estimators': [5, 10, 15, 20, 30, 50], 'loss': ['linear', 'square', 'exponential']}
    },
    'BaggingRegressor': {
        'model': BaggingRegressor(),
        'params': {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 1.0], 'max_features': [0.5, 1.0]}
    },
    'ExtraTreesRegressor': {
        'model': ExtraTreesRegressor(),
        'params': {'n_estimators': [20, 50, 100, 200], 'max_depth': [2, 5, 10, 20], 'min_samples_split': [2, 5, 10]}
    },
    'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(),
        'params': {'n_estimators': [20, 50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5], 'max_depth': [3, 5, 7], 'loss': ['ls', 'lad', 'huber', 'quantile']}
    },
    'CatBoostRegressor': {
        'model': CatBoostRegressor(verbose=False),
        'params': {'iterations': [100, 500], 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0], 'subsample': [0.5, 0.7, 1.0], 'depth': [3, 5, 7]}
    }
}

   
    scores = []
    for algo_name, config in algos.items():  # Используем стандартный цикл for
        gs = GridSearchCV(config['model'], config['params'], cv=10, return_train_score=False, scoring='r2')
        gs.fit(X, y_1)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

In [186]:
res_mupr = my_best_model(X_train, y_train)

In [188]:
res_mupr.sort_values(['best_score'])

,model,best_score,best_params
0,LinearRegression,-17.919547,{}
1,GammaRegressor,-14.156777,{}
2,ARDRegression,-6.943522,{'tol': 0.001}
4,BaggingRegressor,-5.173590,"{'max_features': 0.5, 'max_samples': 1.0, 'n_e..."
3,AdaBoostRegressor,-4.036066,"{'loss': 'linear', 'n_estimators': 5}"
5,ExtraTreesRegressor,-3.861599,"{'max_depth': 10, 'min_samples_split': 5, 'n_e..."
6,GradientBoostingRegressor,-3.782408,"{'learning_rate': 0.5, 'loss': 'huber', 'max_d..."
7,CatBoostRegressor,-3.059801,"{'depth': 5, 'iterations': 500, 'learning_rate..."


In [378]:
res_mupr.sort_values(['best_score']).iloc[-1]['best_params']

{'depth': 5, 'iterations': 500, 'learning_rate': 1.0, 'subsample': 0.5}

**1.LinearRegression**

In [194]:
LR =LinearRegression()
LR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(LR.predict(X_test), y_test))
print('MSE: ', mean_squared_error(LR.predict(X_test), y_test))
print('R2: ', r2_score(LR.predict(X_test), y_test))

MAE:  2.0757283830744035
MSE:  5.753046749622899
R2:  0.1324710277266552


In [196]:
LR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', LR)],
                      verbose=True)

In [200]:
LR_norm.fit(X_train, y_train)

[Pipeline] ............. (step 1 of 2) Processing caler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('caler', MinMaxScaler()), ('model', LinearRegression())],
         verbose=True)

In [202]:
print('MAE: ', mean_absolute_error(LR_norm.predict(X_test), y_test))
print('MSE: ', mean_squared_error(LR_norm.predict(X_test), y_test))
print('R2: ', r2_score(LR_norm.predict(X_test), y_test))

MAE:  2.075728383074407
MSE:  5.753046749622944
R2:  0.13247102772665476


**2.GammaRegressor**

In [207]:
GR =GammaRegressor()
GR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(GR.predict(X_test), y_test))
print('MSE: ', mean_squared_error(GR.predict(X_test), y_test))
print('R2: ', r2_score(GR.predict(X_test), y_test))

MAE:  1.841380783077818
MSE:  4.287038768002264
R2:  0.1554088192024684


In [209]:
GR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', GR)],
                      verbose=True)

In [211]:
GR_norm.fit(X_train, y_train)

[Pipeline] ............. (step 1 of 2) Processing caler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('caler', MinMaxScaler()), ('model', GammaRegressor())],
         verbose=True)

In [213]:
print('MAE: ', mean_absolute_error(GR_norm.predict(X_test), y_test))
print('MSE: ', mean_squared_error(GR_norm.predict(X_test), y_test))
print('R2: ', r2_score(GR_norm.predict(X_test), y_test))

MAE:  2.627401594009283
MSE:  9.006257213252807
R2:  -161.02221946625755


**3.ARDRegression**

In [243]:
ARDR =ARDRegression (tol = 0.001)
ARDR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(ARDR.predict(X_test), y_test))
print('MSE: ', mean_squared_error(ARDR.predict(X_test), y_test))
print('R2: ', r2_score(ARDR.predict(X_test), y_test))

MAE:  2.5933364581830367
MSE:  8.303500640478566
R2:  -8.503116987539054


In [245]:
ARDR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ARDR)],
                      verbose=True)

In [247]:
ARDR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', ARDRegression())],
         verbose=True)

In [249]:
print('MAE: ', mean_absolute_error(ARDR_norm.predict(X_test), y_test))
print('MSE: ', mean_squared_error(ARDR_norm.predict(X_test), y_test))
print('R2: ', r2_score(ARDR_norm.predict(X_test), y_test))

MAE:  1.9065296712920816
MSE:  4.279219387907589
R2:  -0.4923517004086453


**4.BaggingRegressor**

In [271]:
BR = BaggingRegressor (max_features=0.5, max_samples=1.0, n_estimators=10)
BR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(BR.predict(X_test), y_test))
print('MSE: ', mean_squared_error(BR.predict(X_test), y_test))
print('R2: ', r2_score(BR.predict(X_test), y_test))

MAE:  1.8253860796255985
MSE:  4.743810073425188
R2:  -0.3748898983810707


In [273]:
BR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', BR)],
                      verbose=True)

In [275]:
BR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model', BaggingRegressor(max_features=0.5))],
         verbose=True)

In [277]:
print('MAE: ', mean_absolute_error(BR_norm.predict(X_test), y_test))
print('MSE: ', mean_squared_error(BR_norm.predict(X_test), y_test))
print('R2: ', r2_score(BR_norm.predict(X_test), y_test))

MAE:  1.8286482481694897
MSE:  5.7604329731504675
R2:  -2.495782786653333


**5.AdaBoostRegressor**

In [320]:
ABR = AdaBoostRegressor (loss= 'linear', n_estimators = 5 )
ABR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(ABR.predict(X_test), y_test))
print('MSE: ', mean_squared_error(ABR.predict(X_test), y_test))
print('R2: ', r2_score(ABR.predict(X_test), y_test))

MAE:  0.8052288885122086
MSE:  1.1662373320596577
R2:  0.8103137884919843


**Сохраним модель**

In [614]:
file_path = "C:/Users/Владимир/ВКР_Итог/y_pred_1.pkl"
with open(file_path, "wb") as file:
    pickle.dump(ABR, file)

In [328]:
ABR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ABR)],
                      verbose=True)

In [330]:
ABR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model', AdaBoostRegressor(n_estimators=5))],
         verbose=True)

In [332]:
print('MAE: ', mean_absolute_error(ABR_norm.predict(X_test), y_test))
print('MSE: ', mean_squared_error(ABR_norm.predict(X_test), y_test))
print('R2: ', r2_score(ABR_norm.predict(X_test), y_test))

MAE:  1.256314407214429
MSE:  2.370863798326625
R2:  0.5852467634105171


**6.ExtraTreesRegressor**

In [359]:
ETR = ExtraTreesRegressor (max_depth=10, min_samples_split=5, n_estimators=20)
ETR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(ETR.predict(X_test), y_test))
print('MSE: ', mean_squared_error(ETR.predict(X_test), y_test))
print('R2: ', r2_score(ETR.predict(X_test), y_test))

MAE:  1.3538765421135643
MSE:  4.173693860633305
R2:  -0.27253848362548916


In [361]:
ETR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ETR)],
                      verbose=True)

In [363]:
ETR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 ExtraTreesRegressor(max_depth=10, min_samples_split=5,
                                     n_estimators=20))],
         verbose=True)

In [365]:
print('MAE: ', mean_absolute_error(ETR_norm.predict(X_test), y_test))
print('MSE: ', mean_squared_error(ETR_norm.predict(X_test), y_test))
print('R2: ', r2_score(ETR_norm.predict(X_test), y_test))

MAE:  1.646714400892698
MSE:  5.147503305003765
R2:  -0.9204886388327411


**7.GradientBoostingRegressor**

In [386]:
GBR = GradientBoostingRegressor (learning_rate = 0.5, loss = 'huber', max_depth = 7, n_estimators = 200)
GBR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(GBR.predict(X_test), y_test))
print('MSE: ', mean_squared_error(GBR.predict(X_test), y_test))
print('R2: ', r2_score(GBR.predict(X_test), y_test))

MAE:  1.4412778538091082
MSE:  4.718867113777157
R2:  0.2052153850759586


In [388]:
GBR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', GBR)],
                      verbose=True)

In [390]:
GBR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 GradientBoostingRegressor(learning_rate=0.5, loss='huber',
                                           max_depth=7, n_estimators=200))],
         verbose=True)

In [392]:
print('MAE: ', mean_absolute_error(GBR_norm.predict(X_test), y_test))
print('MSE: ', mean_squared_error(GBR_norm.predict(X_test), y_test))
print('R2: ', r2_score(GBR_norm.predict(X_test), y_test))

MAE:  1.7069735968248771
MSE:  8.414222914445464
R2:  -0.14290818947068584


**8.CatBoostRegressor**

In [425]:
CBR = CatBoostRegressor (depth = 5, iterations=500, learning_rate = 1.0, subsample = 0.5, verbose=False)
CBR.fit(X_train, y_train)
print('MAE: ', mean_absolute_error(CBR.predict(X_test), y_test))
print('MSE: ', mean_squared_error(CBR.predict(X_test), y_test))
print('R2: ', r2_score(CBR.predict(X_test), y_test))

MAE:  2.0079796660631266
MSE:  6.980367874385071
R2:  -2.670016295876415


In [427]:
CBR_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', CBR)],
                      verbose=True)

In [429]:
CBR_norm.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x00000145203DB740>)],
         verbose=True)

In [407]:
print('MAE: ', mean_absolute_error(CBR_norm.predict(X_test), y_test))
print('MSE: ', mean_squared_error(CBR_norm.predict(X_test), y_test))
print('R2: ', r2_score(CBR_norm.predict(X_test), y_test))

MAE:  2.0079796660631266
MSE:  6.980367874385071
R2:  -2.670016295876415


Наилучший показатель для переменной **'Модуль упругости при растяжении, ГПа'** показала модель **AdaBoostRegressor** без нормализации, \
для которой **r2 = 0.8103137884919843**.\
Файл с моделью будет сохранен и использован в приложении.

**2. Подбор модели для значения 'Прочность при растяжении, МПа'.**

In [43]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y_2, test_size=0.3, random_state=42)

In [415]:
def my_best_model(X, y_2):
    algos = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}
    },
    'GammaRegressor': {
        'model': GammaRegressor(),
        'params': {}
   },
    'ARDRegression': {
        'model': ARDRegression(),
        'params': {'tol': [1e-3, 1e-2]}
    },
    'AdaBoostRegressor': {
        'model': AdaBoostRegressor(),
        'params': {'n_estimators': [5, 10, 15, 20, 30, 50], 'loss': ['linear', 'square', 'exponential']}
    },
    'BaggingRegressor': {
        'model': BaggingRegressor(),
        'params': {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 1.0], 'max_features': [0.5, 1.0]}
    },
    'ExtraTreesRegressor': {
        'model': ExtraTreesRegressor(),
        'params': {'n_estimators': [20, 50, 100, 200], 'max_depth': [2, 5, 10, 20], 'min_samples_split': [2, 5, 10]}
    },
    'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(),
        'params': {'n_estimators': [20, 50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5], 'max_depth': [3, 5, 7], 'loss': ['ls', 'lad', 'huber', 'quantile']}
    },
    'CatBoostRegressor': {
        'model': CatBoostRegressor(verbose=False),
        'params': {'iterations': [100, 500], 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0], 'subsample': [0.5, 0.7, 1.0], 'depth': [3, 5, 7]}
    }
}

   
    scores = []
    for algo_name, config in algos.items():  # Используем стандартный цикл for
        gs = GridSearchCV(config['model'], config['params'], cv=10, return_train_score=False, scoring='r2')
        gs.fit(X, y_2)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

In [417]:
res_mupr = my_best_model(X_train_2, y_train_2)

In [419]:
res_mupr.sort_values(['best_score'])

,model,best_score,best_params
0,LinearRegression,-10.358839,{}
2,ARDRegression,-5.373563,{'tol': 0.01}
6,GradientBoostingRegressor,-3.616197,"{'learning_rate': 0.01, 'loss': 'huber', 'max_..."
1,GammaRegressor,-2.569186,{}
7,CatBoostRegressor,-2.321341,"{'depth': 7, 'iterations': 100, 'learning_rate..."
3,AdaBoostRegressor,-2.243952,"{'loss': 'exponential', 'n_estimators': 30}"
5,ExtraTreesRegressor,-1.972986,"{'max_depth': 10, 'min_samples_split': 2, 'n_e..."
4,BaggingRegressor,-1.668895,"{'max_features': 0.5, 'max_samples': 1.0, 'n_e..."


In [560]:
res_mupr.sort_values(['best_score']).iloc[-1]['best_params']

{'max_features': 0.5, 'max_samples': 1.0, 'n_estimators': 10}

**1.LinearRegression**

In [463]:
LR_2 =LinearRegression()
LR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(LR_2.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(LR_2.predict(X_test_2), y_test_2))
print('R2: ', r2_score(LR_2.predict(X_test_2), y_test_2))

MAE:  623.7745739312428
MSE:  473638.04396031
R2:  0.04410920759145409


**Сохраним модель**

In [616]:
file_path = "C:/Users/Владимир/ВКР_Итог/y_pred_2.pkl"
with open(file_path, "wb") as file:
    pickle.dump(LR_2, file)

In [465]:
LR_norm_2 = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', LR_2)],
                      verbose=True)

In [467]:
LR_norm_2.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', LinearRegression())],
         verbose=True)

In [485]:
print('MAE: ', mean_absolute_error(LR_norm_2.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(LR_norm_2.predict(X_test_2), y_test_2))
print('R2: ', r2_score(LR_norm_2.predict(X_test_2), y_test_2))

MAE:  623.7745739312403
MSE:  473638.0439603066
R2:  0.044109207591452315


**2.ARDRegression**

In [603]:
ARDR_2 =ARDRegression(tol=0.01)
ARDR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(ARDR_2.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(ARDR_2.predict(X_test_2), y_test_2))
print('R2: ', r2_score(ARDR_2.predict(X_test_2), y_test_2))

MAE:  449.0353786409726
MSE:  276128.6408880153
R2:  -15.984006040077809


In [490]:
ARDR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ARDR_2)],
                      verbose=True)

In [492]:
ARDR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', ARDRegression(tol=0.01))],
         verbose=True)

In [494]:
print('MAE: ', mean_absolute_error(ARDR_2_norm.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(ARDR_2_norm.predict(X_test_2), y_test_2))
print('R2: ', r2_score(ARDR_2_norm.predict(X_test_2), y_test_2))

MAE:  460.2161313249264
MSE:  278366.25421555666
R2:  -20.860818530104282


**3.GradientBoostingRegressor**

In [601]:
GBR_2 =GradientBoostingRegressor(learning_rate=0.01, loss='huber', max_depth = 3, n_estimators=20)
GBR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(GBR_2.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(GBR_2.predict(X_test_2), y_test_2))
print('R2: ', r2_score(GBR_2.predict(X_test_2), y_test_2))

MAE:  426.89290223165216
MSE:  238395.8396007046
R2:  -38.378966579855366


In [500]:
GBR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', GBR_2)],
                      verbose=True)

In [502]:
GBR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 GradientBoostingRegressor(learning_rate=0.01, loss='huber',
                                           n_estimators=20))],
         verbose=True)

In [504]:
print('MAE: ', mean_absolute_error(GBR_2_norm.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(GBR_2_norm.predict(X_test_2), y_test_2))
print('R2: ', r2_score(GBR_2_norm.predict(X_test_2), y_test_2))

MAE:  430.20238461428534
MSE:  242203.44281001543
R2:  -37.64746545295294


**4.GammaRegressor**

In [597]:
GR_2 = GammaRegressor()
GR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(GR_2.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(GR_2.predict(X_test_2), y_test_2))
print('R2: ', r2_score(GR_2.predict(X_test_2), y_test_2))

MAE:  456.87068777590895
MSE:  262844.88434545905
R2:  -1.271039868955386e+30


In [513]:
GR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', GR_2)],
                      verbose=True)

In [515]:
GR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()), ('model', GammaRegressor())],
         verbose=True)

In [517]:
print('MAE: ', mean_absolute_error(GR_2_norm.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(GR_2_norm.predict(X_test_2), y_test_2))
print('R2: ', r2_score(GR_2_norm.predict(X_test_2), y_test_2))

MAE:  450.5641697647366
MSE:  257126.75327577573
R2:  -478.3873494038195


**5.CatBoostRegressor**

In [595]:
CBR_2 =CatBoostRegressor(depth=7, iterations=100, learning_rate=0.1, subsample=0.7,verbose=False)
CBR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(CBR_2.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(CBR_2.predict(X_test_2), y_test_2))
print('R2: ', r2_score(CBR_2.predict(X_test_2), y_test_2))

MAE:  346.86503609742005
MSE:  173018.99184279112
R2:  -7.618113842773479


In [528]:
CBR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', CBR_2)],
                      verbose=True)

In [530]:
CBR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.1s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x000001451FD8FA70>)],
         verbose=True)

In [532]:
print('MAE: ', mean_absolute_error(CBR_2_norm.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(CBR_2_norm.predict(X_test_2), y_test_2))
print('R2: ', r2_score(CBR_2_norm.predict(X_test_2), y_test_2))

MAE:  346.86503609742005
MSE:  173018.99184279112
R2:  -7.618113842773479


**6.AdaBoostRegressor**

In [591]:
ABR_2= AdaBoostRegressor (loss= 'exponential', n_estimators=30)
ABR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(ABR_2.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(ABR_2.predict(X_test_2), y_test_2))
print('R2: ', r2_score(ABR_2.predict(X_test_2), y_test_2))

MAE:  341.9461376266727
MSE:  186183.5370461639
R2:  -4.098905887642678


In [539]:
ABR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ABR_2)],
                      verbose=True)

In [541]:
ABR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model',
                 AdaBoostRegressor(loss='exponential', n_estimators=30))],
         verbose=True)

In [543]:
print('MAE: ', mean_absolute_error(ABR_2_norm.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(ABR_2_norm.predict(X_test_2), y_test_2))
print('R2: ', r2_score(ABR_2_norm.predict(X_test_2), y_test_2))

MAE:  420.26293631811194
MSE:  230471.7507882189
R2:  -3.517256747963148


**7.ExtraTreesRegressor**

In [550]:
ETR_2 =ExtraTreesRegressor(max_depth=10, min_samples_split = 2, n_estimators=20)
ETR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(ETR_2.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(ETR_2.predict(X_test_2), y_test_2))
print('R2: ', r2_score(ETR_2.predict(X_test_2), y_test_2))

MAE:  404.755152319854
MSE:  217654.45181701684
R2:  -2.095491584967547


In [552]:
ETR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', ETR_2)],
                      verbose=True)

In [554]:
ETR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model', ExtraTreesRegressor(max_depth=10, n_estimators=20))],
         verbose=True)

In [558]:
print('MAE: ', mean_absolute_error(ETR_2_norm.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(ETR_2_norm.predict(X_test_2), y_test_2))
print('R2: ', r2_score(ETR_2_norm.predict(X_test_2), y_test_2))

MAE:  402.4427633476192
MSE:  217700.02729511738
R2:  -8.322000037586047


**8.BaggingRegressor**

In [563]:
BR_2 = BaggingRegressor(max_features=0.5, max_samples= 1.0, n_estimators=10)
BR_2.fit(X_train_2, y_train_2)
print('MAE: ', mean_absolute_error(BR_2.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(BR_2.predict(X_test_2), y_test_2))
print('R2: ', r2_score(BR_2.predict(X_test_2), y_test_2))

MAE:  283.4856441932064
MSE:  130370.73173089215
R2:  -1.3356346255796447


In [565]:
BR_2_norm = Pipeline(steps=[('Scaler', MinMaxScaler ()),
                      ('model', BR_2)],
                      verbose=True)

In [567]:
BR_2_norm.fit(X_train_2, y_train_2)

[Pipeline] ............ (step 1 of 2) Processing Scaler, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('model', BaggingRegressor(max_features=0.5))],
         verbose=True)

In [569]:
print('MAE: ', mean_absolute_error(BR_2_norm.predict(X_test_2), y_test_2))
print('MSE: ', mean_squared_error(BR_2_norm.predict(X_test_2), y_test_2))
print('R2: ', r2_score(BR_2_norm.predict(X_test_2), y_test_2))

MAE:  405.779450049187
MSE:  235881.56018927423
R2:  -7.1765592316999225


Наилучший показатель для переменной **'Прочность при растяжении, МПа'** показала модель **LinearRegression** без нормализации,
для которой **R2:  0.04410920759145409**.\
Файл с моделью будет сохранен и использован в приложении.